
# Dashboard Interactivo — Importaciones y Consumos de Combustibles (Avance)

**Herramientas:** Voilá + Plotly + ipywidgets  
**Paleta:** Verde `#2E6E36`, Amarillo `#FFB81C`, Naranja `#E35205`, Azul `#003366` (fondo claro `#E6EEF3`)  


**Ignacio Méndez Alvarez 22613 y Diego Soto Flores 22737**

Enlace a GitHub: https://github.com/ignaciomendeza/DS-Lab-11.git

In [1]:

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display, HTML


from sklearn.model_selection import train_test_split, learning_curve
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import pandas as _pd
import math
import ipywidgets as _widgets
from IPython.display import display as _display, HTML as _HTML
import plotly.io as pio

FONDO = "#E6EEF3"
PALETA = {
    "Gasolina regular": "#2E6E36",
    "Gasolina superior": "#FFB81C",
    "Diesel alto azufre": "#E35205",
    "AzulOscuro": "#003366"
}

def apply_bg(fig):
    fig.update_layout(plot_bgcolor=FONDO, paper_bgcolor=FONDO)
    return fig


In [2]:

importacion = pd.read_excel("Datos.xlsx", sheet_name= "IMPORTACION", header=6) 
importacion = importacion.iloc[:288].reset_index(drop=True)

consumo = pd.read_excel("Datos.xlsx", sheet_name= "CONSUMO", header=6) 
consumo = consumo.iloc[:300].reset_index(drop= True)

importaciones = importacion[["Fecha", "Gasolina regular", "Gasolina superior", "Diesel alto azufre"]].copy()
consumos = consumo[["Fecha", "Gasolina regular", "Gasolina superior", "Diesel alto azufre"]].copy()

for df in (importaciones, consumos):
    df["Fecha"] = pd.to_datetime(df["Fecha"])
    df["Año"] = df["Fecha"].dt.year
    df["Mes_num"] = df["Fecha"].dt.month
    df["Mes"] = df["Fecha"].dt.month_name()
    df.sort_values("Fecha", inplace=True)

YEARS_ALL = sorted(list(set(importaciones["Año"].unique()).union(consumos["Año"].unique())))
MESES_ORD = list(range(1,13))
NOMBRES_MESES = ['January','February','March','April','May','June','July','August','September','October','November','December']

display(HTML(f"<b>Importaciones:</b> {importaciones.shape[0]} filas — <b>Consumos:</b> {consumos.shape[0]} filas"))


In [3]:

    # === Filtros ===
    dataset_toggle = widgets.ToggleButtons(
        options=[("Importaciones","importaciones"), ("Consumos","consumos")],
        value="importaciones",
        description="Datos:"
    )

    year_dropdown = widgets.Dropdown(
        options=YEARS_ALL,
        value=min(YEARS_ALL),
        description="Año:"
    )

    combustibles_sel = widgets.SelectMultiple(
        options=["Gasolina regular","Gasolina superior","Diesel alto azufre"],
        value=("Gasolina regular","Gasolina superior","Diesel alto azufre"),
        description="Combustibles:",
        rows=3
    )

    mes_dropdown = widgets.Dropdown(
        options=[(NOMBRES_MESES[m-1], m) for m in MESES_ORD],
        value=1,
        description="Mes:"
    )

    year_range = widgets.IntRangeSlider(
        value=(min(YEARS_ALL), max(YEARS_ALL)),
        min=min(YEARS_ALL), max=max(YEARS_ALL), step=1,
        description='Rango años:',
        continuous_update=False
    )

    display(widgets.HBox([dataset_toggle, year_dropdown, combustibles_sel, mes_dropdown]))
    display(year_range)


IntRangeSlider(value=(2000, 2024), continuous_update=False, description='Rango años:', max=2024, min=2000)

In [4]:

def current_df():
    return importaciones if dataset_toggle.value == "importaciones" else consumos


In [5]:

def viz1_barras_mensuales(año, combustibles):
    df = current_df()
    tmp = (df[df["Año"]==año]
           .groupby("Mes_num")[list(combustibles)].mean()
           .reindex(range(1,13))
           .reset_index())
    tmp["Mes"] = [NOMBRES_MESES[m-1] for m in tmp["Mes_num"]]

    fig = go.Figure()
    for c in combustibles:
        fig.add_bar(x=tmp["Mes"], y=tmp[c], name=c, marker_color=PALETA[c])
    fig.update_layout(
        title=f"Promedio mensual por tipo — {año} ({'Importaciones' if dataset_toggle.value=='importaciones' else 'Consumos'})",
        barmode="group", xaxis_title="Mes", yaxis_title="Volumen"
    )
    return apply_bg(fig)


In [6]:

def viz2_pie_share(año):
    df = current_df()
    cols = ["Gasolina regular","Gasolina superior","Diesel alto azufre"]
    vals = df[df["Año"]==año][cols].mean()
    fig = px.pie(values=vals.values, names=vals.index, color=vals.index,
                 color_discrete_map=PALETA,
                 title=f"Distribución porcentual — {año}")
    return apply_bg(fig)


In [7]:

def viz3_serie_tiempo(year_min, year_max, combustibles):
    df = current_df()
    tmp = df[(df["Año"]>=year_min) & (df["Año"]<=year_max)].copy()
    fig = go.Figure()
    for c in combustibles:
        fig.add_scatter(x=tmp["Fecha"], y=tmp[c], mode="lines", name=c, line=dict(width=2.5, color=PALETA[c]))
    fig.update_layout(title=f"Tendencia temporal {year_min}-{year_max}", xaxis_title="Fecha", yaxis_title="Volumen")
    return apply_bg(fig)


In [8]:

import plotly.io as pio
def viz4_heatmap(col):
    df = current_df()
    pivot = (df.groupby(["Año","Mes_num"])[col].mean()
               .unstack(fill_value=0).reindex(index=YEARS_ALL, columns=MESES_ORD))
    fig = px.imshow(pivot, aspect="auto", labels=dict(x="Mes", y="Año", color="Promedio"),
                    x=[NOMBRES_MESES[m-1] for m in MESES_ORD], y=YEARS_ALL,
                    title=f"Heatmap mensual por año — {col}")
    return apply_bg(fig)

combustible_heatmap = widgets.Dropdown(
    options=["Gasolina regular","Gasolina superior","Diesel alto azufre"],
    value="Gasolina regular",
    description="Heatmap:"
)
display(combustible_heatmap)


Dropdown(description='Heatmap:', options=('Gasolina regular', 'Gasolina superior', 'Diesel alto azufre'), valu…

In [10]:
pio.renderers.default = "notebook_connected"

fuel_model_select = widgets.Dropdown(
    options=["Gasolina regular","Gasolina superior","Diesel alto azufre"],
    value="Gasolina regular",
    description="Combustible:"
)

models_options = {
    "LinearRegression": LinearRegression(),
    "RandomForest": RandomForestRegressor(n_estimators=100, random_state=42),
    "GradientBoosting": GradientBoostingRegressor(n_estimators=100, random_state=42)
}

models_sel = widgets.SelectMultiple(
    options=list(models_options.keys()),
    value=("LinearRegression","RandomForest"),
    description="Modelos:",
    rows=3
)

train_button = widgets.Button(description="Entrenar & Mostrar", button_style="primary")


In [11]:
out_models_panel = widgets.Output()
out_models_table = widgets.Output()
out_learning = widgets.Output()
out_pred_scatter = widgets.Output()
out_residuals = widgets.Output()

In [12]:
def prepare_features(df, fuel_col):
    tmp = df[[ "Año", "Mes_num", fuel_col]].dropna().copy()
    X = tmp[["Año","Mes_num"]].values
    y = tmp[fuel_col].values
    return X, y, tmp

def train_and_eval(fuel_col):
    df = current_df()  
    X, y, tmp = prepare_features(df, fuel_col)
    if len(y) < 10:
        raise ValueError("Datos insuficientes para entrenar (menos de 10 filas).")
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)
    scaler = StandardScaler()
    X_train_s = scaler.fit_transform(X_train)
    X_test_s = scaler.transform(X_test)

    results = {}
    fitted_models = {}
    for name, model in models_options.items():
        m = model
        m.fit(X_train_s, y_train)
        y_pred = m.predict(X_test_s)
        mae = mean_absolute_error(y_test, y_pred)
        rmse = math.sqrt(mean_squared_error(y_test, y_pred))
        r2 = r2_score(y_test, y_pred)
        results[name] = {"mae": mae, "rmse": rmse, "r2": r2, "y_test": y_test, "y_pred": y_pred, "X_test": X_test}
        fitted_models[name] = (m, scaler)
    return results, fitted_models, (X_train, X_test, y_train, y_test, scaler, tmp)

In [13]:
# Entrenamiento de modelo
def compute_learning_curve(model, X, y):
    train_sizes, train_scores, test_scores = learning_curve(model, X, y, cv=5,
                                                            train_sizes=[0.2,0.4,0.6,0.8,1.0],
                                                            scoring='neg_mean_squared_error', n_jobs=1)
    train_rmse = [math.sqrt(-s.mean()) for s in train_scores]
    test_rmse = [math.sqrt(-s.mean()) for s in test_scores]
    return train_sizes, train_rmse, test_rmse

def plot_model_comparison_table(results, selected_models):
    rows = []
    for m in selected_models:
        r = results[m]
        rows.append({"Modelo": m, "MAE": r["mae"], "RMSE": r["rmse"], "R2": r["r2"]})
    dfm = pd.DataFrame(rows).set_index("Modelo")
    with out_models_table:
        out_models_table.clear_output(wait=True)
        _display(_HTML("<h4>Comparativa de desempeño</h4>"))
        _display(dfm.style.format("{:.3f}"))

In [14]:
# Predicción con modelos
def plot_prediction_scatter(results, selected_models):
    if len(selected_models) == 0:
        return
    m0 = selected_models[0]
    y_test = results[m0]["y_test"]
    y_pred = results[m0]["y_pred"]
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=y_test, y=y_pred, mode="markers", name=f"{m0} pred vs real"))
    fig.add_trace(go.Line(x=[min(y_test), max(y_test)], y=[min(y_test), max(y_test)], name="y=x", line=dict(dash='dot')))
    fig.update_layout(title=f"Predicción vs Real — {m0}", xaxis_title="Real", yaxis_title="Predicho")
    fig = apply_bg(fig)
    with out_pred_scatter:
        out_pred_scatter.clear_output(wait=True)
        display(fig)

def plot_residuals_hist(results, selected_models):
    if len(selected_models) == 0:
        return
    m0 = selected_models[0]
    y_test = results[m0]["y_test"]
    y_pred = results[m0]["y_pred"]
    residuals = y_test - y_pred
    fig = px.histogram(residuals, nbins=20, labels={"value":"Residual"}, title=f"Residuals histogram — {m0}")
    fig = apply_bg(fig)
    with out_residuals:
        out_residuals.clear_output(wait=True)
        display(fig)

def plot_learning_curve_for_models(results, fitted_models, X_all, y_all, selected_models):
    with out_learning:
        out_learning.clear_output(wait=True)
        for mname in selected_models:
            model = models_options[mname]
            try:
                ts, train_rmse, test_rmse = compute_learning_curve(model, X_all, y_all)
            except Exception as e:
                print(f"Error en learning_curve para {mname}: {e}")
                continue
            fig = go.Figure()
            fig.add_trace(go.Scatter(x=ts, y=train_rmse, mode="lines+markers", name="Train RMSE"))
            fig.add_trace(go.Scatter(x=ts, y=test_rmse, mode="lines+markers", name="Test RMSE"))
            fig.update_layout(title=f"Learning curve (RMSE) — {mname}", xaxis_title="Train size", yaxis_title="RMSE")
            display(apply_bg(fig))

In [15]:
def on_train_button_clicked(b):
    out_models_panel.clear_output(wait=True)
    try:
        fuel = fuel_model_select.value
        results, fitted_models, (X_train, X_test, y_train, y_test, scaler, tmp) = train_and_eval(fuel)
    except Exception as e:
        with out_models_panel:
            print("Error entrenando modelos:", e)
        return

    selected_models = list(models_sel.value)

    with out_models_panel:
        out_models_panel.clear_output(wait=True)
        _display(_HTML(f"<h3 style='color:{PALETA['AzulOscuro']}'>Modelado: {fuel}</h3>"))
        display(widgets.HBox([_HTML("<b>Modelos seleccionados:</b>"), _HTML(", ".join(selected_models))]))

        plot_model_comparison_table(results, selected_models)

        plot_prediction_scatter(results, selected_models)

        plot_residuals_hist(results, selected_models)

        plot_learning_curve_for_models(results, fitted_models, _pd.concat([_pd.DataFrame(X_train, columns=["Año","Mes_num"]), _pd.DataFrame(X_test, columns=["Año","Mes_num"])]).values,
                                      _pd.concat([_pd.Series(y_train), _pd.Series(y_test)]).values, selected_models)

train_button.on_click(on_train_button_clicked)

display(widgets.HBox([fuel_model_select, models_sel, train_button]))
display(out_models_panel)


Output()

In [9]:

out1 = widgets.Output()
out2 = widgets.Output()
out3 = widgets.Output()
out4 = widgets.Output()

def render_all(*args):
    out1.clear_output(wait=True)
    out2.clear_output(wait=True)
    out3.clear_output(wait=True)
    out4.clear_output(wait=True)
    with out1:
        display(viz1_barras_mensuales(year_dropdown.value, combustibles_sel.value))
    with out2:
        display(viz2_pie_share(year_dropdown.value))
    with out3:
        yr0, yr1 = year_range.value
        display(viz3_serie_tiempo(yr0, yr1, combustibles_sel.value))
    with out4:
        display(viz4_heatmap(combustible_heatmap.value))

for w in (dataset_toggle, year_dropdown, combustibles_sel, mes_dropdown, year_range, combustible_heatmap):
    w.observe(render_all, names="value")

display(
    widgets.VBox([
        widgets.HTML(f"<h3 style='color:{PALETA['AzulOscuro']}'>Dashboard — Avance</h3>"),
        widgets.HBox([out1, out2]),
        widgets.HBox([out3]),
        widgets.HBox([out4])
    ])
)

render_all()
